# Trabalho Final de Plataformas cognitivas

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
import os

from google.cloud import bigquery

In [ ]:
#@title Informe o id do projeto nesta linha e o bucket. Sugestão, coloque o nome do bucket como nome do projeto + um sufixo:
PROJECT = 'trabalho-final-plat-cog-fiap' #@param {type:"string"}
BUCKET = 'trabalho-final-plat-cog-fiap-bucket' #@param {type:"string"}

In [ ]:
#Outras variáveis:
REGION = 'us-central1'

#Variáveis do OS
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION

In [ ]:
%%bash
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: trabalho-final-plat-cog-fiap


In [ ]:
%%bash

# Cria um BigQuery dataset
datasetexists=$(bq ls -d --project_id=$PROJECT | grep -w analise_credito)

if [ -n "$datasetexists" ]; then
    echo -e "BigQuery dataset já existe. Show, não vamos precisar criá-lo."

else
    echo "Criando BigQuery dataset chamado: analise_credito"
    
    bq --location=US mk --dataset \
        --description "Dados de Crédito Pessoal" \
        $PROJECT:analise_credito
fi
echo "Estes são os datasets do projeto:"
bq ls -d --project_id=$PROJECT   

BigQuery dataset já existe. Show, não vamos precisar criá-lo.
Estes são os datasets do projeto:
     datasetId     
 ----------------- 
  analise_credito  
  pesobebes        


In [ ]:
%%bash
## Cria um GCS bucket se não existir already...
exists=$(gsutil ls -d -p ${PROJECT}  | grep -w gs://${BUCKET}/)

if [ -n "$exists" ]; then
    echo -e "Bucket já existe. Show, não vamos precisar criá-lo."
    
else
    echo "Criando um novo GCS bucket."
    gsutil mb -l ${REGION} -p ${PROJECT} gs://${BUCKET}
fi
echo "Estes são os buckets do projeto:"
gsutil ls -p ${PROJECT}

Criando um novo GCS bucket.
Estes são os buckets do projeto:
gs://trabalho-final-plat-cog-fiap-bucket/


Creating gs://trabalho-final-plat-cog-fiap-bucket/...


In [ ]:
%%bigquery --project {PROJECT}
CREATE OR REPLACE TABLE
    analise_credito.analise_credito AS
SELECT
* 
FROM
    emf-teacher.trabalho_loans.loan_default

Query is running:   0%|          |

""


In [ ]:
# Criamos um BigQuery client object.
bqclient=bigquery.Client(project=PROJECT)

In [ ]:
dataset_name = "analise_credito"

# Criamos um objeto de referência para o dataset
dataset_ref = bqclient.dataset( dataset_id=dataset_name, project=bqclient.project)

# Export ambas tabelas train and eval tables
destination_uri = os.path.join(
        "gs://", BUCKET, dataset_name, "data", "raw.csv")
table_name = "analise_credito"
table_ref = dataset_ref.table(table_name)
extract_job = bqclient.extract_table(
        table_ref,
        destination_uri,
        # Local do bucket deve ser o mesmo do dataset
        location="US",
    )  # API request
extract_job.result()  # Espera a exportação acontecer.

print("Exported {}:{}.{} to {}".format(
        bqclient.project, dataset_name, table_name, destination_uri))

Exported trabalho-final-plat-cog-fiap:analise_credito.analise_credito to gs://trabalho-final-plat-cog-fiap-bucket/analise_credito/data/raw.csv
